<a href="https://colab.research.google.com/github/pu-bi/AI-industry-job-experience-for-non-majors/blob/main/4-HTML-CSS-JS/Flask_web_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 저장한 모델을 불러오기 위해 구글 드라이브에 마운트합니다.
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# HuggingFace Transformers 라이브러리를 설치합니다.
!pip install -q transformers

     |████████████████████████████████| 5.5 MB 5.4 MB/s 
     |████████████████████████████████| 7.6 MB 49.7 MB/s 
     |████████████████████████████████| 182 kB 30.4 MB/s 


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
import torch
import torch.nn.functional as F
import numpy as np

# 모델 불러오기 : 본인의 my_model 폴더 경로를 입력하세요.
model_path = "/content/drive/MyDrive/가을_중앙정보처리학원/my_model"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
trainer = Trainer(model=model)
tokenizer = AutoTokenizer.from_pretrained(model_path)
id2label = {0: '활동', 1: '숙소', 2: '맛집', 3: '날씨', 4: '카페', 5: '인사', 6: '관광지추천', 7: '소개'}

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


In [ ]:
def inference(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)
  logits = outputs.logits

  probs = logits.squeeze().cpu()
  predictions = np.where(probs==probs.max(), 1, 0)
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

  return predicted_labels[0]

In [ ]:
# softmax 와 threshold 를 사용하는 새로운 인퍼런스 함수
def inference_with_softmax(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits

  probs = logits.squeeze().cpu()
  softmax_probs = F.softmax(probs, dim=-1)
  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]
  print('입력 텍스트 :', text)
  print('예측 결과 :', id2label[labels_predict])
  print('prob :', softmax_probs[labels_predict])
  if softmax_probs[labels_predict] < 0.6:
    return '기타'
  else:
    return id2label[labels_predict]

In [ ]:
import random

def answer_sentence(pred):

  if pred == '날씨':
      date= ['오늘', '내일', '어제', '그저께', '내일 모레']
      choice_date = random.choice(date)

      weather_2 = ['추워요.', '따뜻해요.', '조금 쌀쌀해요.', '비가 올 것 같아요.' ,'태풍이 와요.', '바람이 불어요.', '더워요.']
      choice_weather_2= random.choice(weather_2)

      attractions_2= ['제주시', '서귀포시', '애월읍', '안덕면', '성산읍', '조천읍', '구좌읍', '한림읍', '남원읍']
      choice_attractions_2 = random.choice(attractions_2)

      sentence = f'{choice_date} {choice_attractions_2}의 날씨는 {choice_weather_2}'

  elif pred == '관광지추천':
 
    attraction = ['쇠소깍', '용머리해안', '함덕 해수욕장', '우도']
    choice_attraction = random.choice(attraction)
 
    nice = ['경치 좋은', '바다가 예쁜', '단체로 갈 만한', '사진찍기 좋은']
    choice_nice = random.choice(nice)
 
    sentence = f'제주 {choice_nice} {choice_attraction}에 가 보세요~'


  elif pred == '숙소':
    accomodation = ['베니키아 호텔', '위드 스테이 호텔 제주', '제주 라온 호텔', '신신호텔 천지연']
    choice_accomodation = random.choice(accomodation)

    location = ['애월에 있는', '제주시에 있는', '한림에 있는', '서귀포에 있는']
    choice_location = random.choice(location)

    sentence = f'제주도 {choice_location} {choice_accomodation} 가 보시는 건 어떠세요?'
  
  elif pred == '맛집':

    restaurant = ['숙성도', '빵귿', '살찐 고등어', '복집식당']
    choice_restaurant = random.choice(restaurant)

    delicious = ['삼겹살이 맛있는', '빵이 맛있는', '고등어가 유명한', '갈치구이 유명한']
    choice_delicious = random.choice(delicious)
    sentence = f'제주 {choice_delicious}, {choice_restaurant} 어떠세요?'
  
  elif pred=='인사':
    bot_name = ['제주심심이', '감귤맨', '제주이루다', '감귤 Killer', '제주시리', '제주스', '코난']
    choice_name = random.choice(bot_name)
    bot_role = ['당신의 친구', '제주의 수호자', '당신의 친절한 가이드', '든든한 도우미']
    choice_role = random.choice(bot_role)
    end_of_sentence = ['입니다.', '랍니다.', '라구요.', '...일까요?']
    choice_eos = random.choice(end_of_sentence)
    sentence = f"제 이름은 {choice_name}, {choice_role}{choice_eos}"
    if choice_name == '제주스':
      sentence = f"내 이름은 {choice_name}, 축구선수는 아니야."
    elif choice_name == '코난':
      sentence = f"내 이름은 {choice_name}, 감귤이죠."
      


  elif pred == '소개':
    sentence = '저는 제주도의 <날씨>,<숙소>,<맛집>,<활동>,<카페> 기능에 관하여 이용이 가능합니다.'

  elif pred == "활동":

    act_1 = ["인기 많은","즐거운","재밌는"]
    choice_act_1 = random.choice(act_1)

    act_2 =["둘레길 걷기","오름 방문","바닷길 드라이브"]
    choice_act_2 = random.choice(act_2)
    sentence = f"{choice_act_1}, {choice_act_2} 해보는 건 어때요?"

  elif pred == '카페':
    cafe=['제주특별자치도 제주시 한경면 낙원로 32 에 있는 산노루','제주특별자치도 서귀포시 이어도로1027번길 34에 있는 카페 귤 꽃다락','제주특별자치도 서귀포시 516로 717에 있는 서귀다원','제주특별자치도 제주시 은수길 110 2F에 있는 우연못']
    choice_cafe = random.choice(cafe)
    ending=['어떠세요?','가보세요~!','가보는걸 추천 드립니다!','추천 합니다.']
    choice_ending=random.choice(ending)
    sentence = f'{choice_cafe} {choice_ending}'
  
  else:
    sentence = "죄송한데, 무슨 말씀인지 잘 모르겠어요."
  
  return sentence

In [ ]:
def answer_system(input_text):
  
  # print("안녕하세요, 무엇을 도와드릴까요?")
  pred = inference_with_softmax(input_text)
  sentence = answer_sentence(pred)
  return sentence 

In [ ]:
answer = answer_system("날씨")
print(answer)

안녕하세요, 무엇을 도와드릴까요?
입력 텍스트 : 날씨
예측 결과 : 날씨
prob : tensor(0.9996, grad_fn=<SelectBackward0>)
어제, 안덕면의 날씨는 비가 올 것 같아요.


In [ ]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [ ]:
# 다음의 명령어로 ngrok linux version 을 설치

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-11-25 07:24:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  5.37MB/s    in 2.4s    

2022-11-25 07:24:05 (5.37 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]



In [ ]:
!ls

drive  ngrok-stable-linux-amd64.tgz  sample_data  tmp_trainer


In [ ]:
# 압축 해제

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [ ]:
# ngrok 사이트 (https://dashboard.ngrok.com/get-started/setup)에서 회원가입을 한뒤
# "Your Authtoken" 탭을 클릭하여 authtoken을 복사하여 붙여넣습니다.

!./ngrok authtoken 2AobOWreg1Cj5sYHFJ4rCPyQBt4_6HVVnb8nMQ4MFvKRzePCi

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# 플라스크 임포트
from flask import Flask
from flask import render_template, request
from flask_ngrok import run_with_ngrok

In [ ]:
from flask import Flask, url_for, redirect, render_template, request

TEMPLATE_FOLDER = '/content/drive/MyDrive/가을_중앙정보처리학원/06/templates'
STATIC_FOLDER = '/content/drive/MyDrive/가을_중앙정보처리학원/06/static'
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat') 
def chatbot():
    return render_template('chat.html')

@app.route('/get')
def get_bot_response():
    userText = request.args.get('a')
    return answer_system(userText)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e81a-35-239-188-124.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:17] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/photo-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/js/slideshow.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/photo-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/photo-3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/img-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/img-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/sns-2.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] "GET /static/images/sns-1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:54:18] 

안녕하세요, 무엇을 도와드릴까요?
입력 텍스트 : 흑돼지 사줘
예측 결과 : 소개
prob : tensor(0.9999, grad_fn=<SelectBackward0>)


...

안녕하세요, 무엇을 도와드릴까요?
입력 텍스트 : 너 바보니?
예측 결과 : 인사
prob : tensor(0.9999, grad_fn=<SelectBackward0>)


INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:56:40] "GET /get?a=숙소 HTTP/1.1" 200 -


안녕하세요, 무엇을 도와드릴까요?
입력 텍스트 : 숙소
예측 결과 : 숙소
prob : tensor(0.9999, grad_fn=<SelectBackward0>)


INFO:werkzeug:127.0.0.1 - - [25/Nov/2022 07:56:45] "GET /get?a=다른%20덴%20없냐 HTTP/1.1" 200 -


안녕하세요, 무엇을 도와드릴까요?
입력 텍스트 : 다른 덴 없냐
예측 결과 : 인사
prob : tensor(0.8888, grad_fn=<SelectBackward0>)
